# Notebook III - Climatic Constraints

<hr>
This module uses various yield reduction factors to apply the obtained maximum attainable yield from Module 2 to consider the constraint effects separately from Module II. This modules works with two separate python look-up scripts ALL_REDUCTION_FACTORS_IRR and ALL_REDUCTION_FACTORS_RAIN which must be manually edited for crop/LUT specific reduction factors.

Prepared by Geoinformatics Center, AIT
<hr>

### Google drive connection
In this step, we will connect to Google Drive service and mount the drive where we will start our PyAEZ project

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Then, installing any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [ ]:
# 'Installing neccessary packages'
# !pip install gdal
# # !pip install pyaez==2.1.0

## Importing Libraries

In [ ]:
'''import supporting libraries'''
# import pyaez
import matplotlib.pyplot as plt
import numpy as np
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys

Setting the working directory -- where our PyAEZ project is located.

In [ ]:
'Set the working directory'
# work_dir = r'/content/gdrive/MyDrive/PyAEZv2/'
work_dir = r'D:\PyAEZv2.1_Draft'
os.chdir(work_dir)
sys.path.append('./pyaez/')
!pwd

## Applying Climate Constraints

In [ ]:
'''importing library'''

from pyaez import ClimaticConstraints
clim_con = ClimaticConstraints.ClimaticConstraints()

from pyaez import UtilitiesCalc
obj_utilities = UtilitiesCalc.UtilitiesCalc()

In [ ]:
'''Reading data'''
lgp = gdal.Open(r'.\data_output\NB1\LAO_LGP.tif').ReadAsArray()
lgp_equv = gdal.Open(r'.\data_output\NB1\LAO_LGPEquivalent.tif').ReadAsArray()
lgp10 = gdal.Open(r'.\data_output\NB1\LAO_LGPt10.tif').ReadAsArray()
tclimate = gdal.Open(r'.\data_output\NB1\LAO_ThermalClimate.tif').ReadAsArray()

max_temp = np.load(r'./data_input/climate/max_temp.npy')# maximum temperature (Daily or Monthly)
min_temp = np.load(r'./data_input/climate/min_temp.npy')  # minimum temperature (Daily or Monthly)

yield_map_rain = gdal.Open(r'.\data_output\NB2\maiz_yield_rain.tif').ReadAsArray()
yield_map_irr = gdal.Open(r'.\data_output\NB2\maiz_yield_irr.tif').ReadAsArray()

In [ ]:
'''Applying climatic constraints. Note that constraints for rainfed and irrigated conditions are run separately'''
# Yield Reduction for Rainfed Conditions
clim_con.applyClimaticConstraints(min_temp = min_temp,
                                  max_temp = max_temp,
                                  lgp= lgp, lgp_equv = lgp_equv,
                                  lgpt10 = lgp10,
                                  yield_input = yield_map_rain,
                                  irr_or_rain = 'rain',
                                  tclimate= tclimate,
                                  no_tclimate = [2,6,7,8,9,10,11,12]) # for maize

# Getting the climate adjusted yield (rainfed)
clim_yield_rain_maiz = clim_con.getClimateAdjustedYield()

# Getting agro-climatic constraint factor (fc3) (rainfed)
fc3_maiz_rain = clim_con.getClimateReductionFactor()

In [ ]:
'''visualize results'''
plt.figure(1, figsize=(22,9))
plt.subplot(1,3,1)
plt.imshow(yield_map_rain, vmax = np.max([clim_yield_rain_maiz, yield_map_rain]))
plt.colorbar(shrink=0.6)
plt.title('Original Rainfed Yield Maize')

plt.subplot(1,3,2)
plt.imshow(clim_yield_rain_maiz, vmax = np.max([clim_yield_rain_maiz, yield_map_rain]))
plt.colorbar(shrink=0.6)
plt.title('Climate Constrainted Rainfed Yield Maize')

plt.subplot(1,3,3)
plt.imshow(fc3_maiz_rain, vmax = 1)
plt.colorbar(shrink=0.6)
plt.title('Fc3 Maize Rainfed')


In [ ]:
'''save result (Rainfed Condition)'''
obj_utilities.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB3\clim_maiz_yield_rain.tif', clim_yield_rain_maiz)
obj_utilities.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB3\fc3_maiz_rain.tif', fc3_maiz_rain)

In [ ]:
'''Applying climatic constraints. Note that constraints for rainfed and irrigated conditions are run separately'''
# Yield Reduction for Rainfed Conditions
clim_con.applyClimaticConstraints(min_temp = min_temp,
                                  max_temp = max_temp,
                                  lgp= lgp, lgp_equv = lgp_equv,
                                  lgpt10 = lgp10,
                                  yield_input = yield_map_irr,
                                  irr_or_rain = 'irr',
                                  tclimate= tclimate,
                                  no_tclimate = [2,6,7,8,9,10,11,12]) # for maize

# Getting the climate adjusted yield (rainfed)
clim_yield_irr_maiz = clim_con.getClimateAdjustedYield()

# Getting agro-climatic constraint factor (fc3) (rainfed)
fc3_maiz_irr = clim_con.getClimateReductionFactor()

In [ ]:
'''visualize results'''
plt.figure(1, figsize=(22,9))
plt.subplot(1,3,1)
plt.imshow(yield_map_irr, vmax = np.max([clim_yield_irr_maiz, yield_map_irr]))
plt.colorbar(shrink=0.6)
plt.title('Original Irrigated Yield Maize')

plt.subplot(1,3,2)
plt.imshow(clim_yield_irr_maiz, vmax = np.max([clim_yield_irr_maiz, yield_map_irr]))
plt.colorbar(shrink=0.6)
plt.title('Climate Constrainted Irrigated Yield Maize')

plt.subplot(1,3,3)
plt.imshow(fc3_maiz_rain, vmax = 1)
plt.colorbar(shrink=0.6)
plt.title('Fc3 Maize Irrigated')


In [ ]:
'''save result (Irrigated Condition)'''
obj_utilities.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB3\clim_maiz_yield_irr.tif', clim_yield_irr_maiz)
obj_utilities.saveRaster(r'.\data_input\LAO_Admin.tif', r'.\data_output\NB3\fc3_maiz_irr.tif', fc3_maiz_irr)

<hr>

### END OF MODULE 3: CLIMATIC CONSTRAINTS

<hr>